##  ewf-ext-03-04-01 - Temperature and Chlorophyll anomalies for coastal areas

### <a name="service">Service definition

In [1]:
service = dict([('title', 'ewf-ext-03-04-01 - Temperature and Chlorophyll anomalies for coastal areas'),
                ('abstract', 'ewf-ext-03-04-01 - Temperature and Chlorophyll anomalies for coastal areas'),
                ('id', 'ewf-ext-03-04-01')])

### Parameter Definition

In [2]:
yoi = dict([('id', 'yoi'),
            ('value', '2019'),
            ('title', 'year of interest'),
            ('abstract', 'year of interest')])

In [3]:
start_year = dict([('id', 'start_year'),
            ('value', '2018'),
            ('title', 'start year'),
            ('abstract', 'start year')])

In [4]:
end_year = dict([('id', 'end_year'),
            ('value', '2019'),
            ('title', 'end_year'),
            ('abstract', 'end_year')])

In [5]:
region_of_interest = dict([('id', 'regionOfInterest'),
                         ('value', 'POLYGON ((0.601779726018505 42.8075056025504,5.06836475455413 42.8075056025504,5.06836475455413 39.2520150325718,0.601779726018505 39.2464595785562,0.601779726018505 42.8075056025504))'),
                         ('title', 'WKT Polygon for the Region of Interest'),
                         ('abstract', 'Set the value of WKT Polygon')])

In [6]:
area_of_interest = dict([('id', 'areaOfInterest'),
                         ('value', 'europe'),
                         ('title', 'Area of the region'),
                         ('abstract', 'Area of the region of interest')])

In [7]:
image_variable = dict([('id', 'imageVariable'),
                         ('value', 'sea_surface_temperature'),
                         ('title', 'Image Variable'),
                         ('abstract', 'Image Variable of Interest')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the Sentinel-1 stack of products' identifiers

In [8]:
input_identifiers = ('20180101000000-GOS-L3S_GHRSST-SSTsubskin-night_SST_HR_NRT-MED-v02.0-fv01.0.nc', '20190101000000-GOS-L3S_GHRSST-SSTsubskin-night_SST_HR_NRT-MED-v02.0-fv01.0.nc',
                      '20180102000000-GOS-L3S_GHRSST-SSTsubskin-night_SST_HR_NRT-MED-v02.0-fv01.0.nc', '20190102000000-GOS-L3S_GHRSST-SSTsubskin-night_SST_HR_NRT-MED-v02.0-fv01.0.nc',
                      '20180103000000-GOS-L3S_GHRSST-SSTsubskin-night_SST_HR_NRT-MED-v02.0-fv01.0.nc', '20190103000000-GOS-L3S_GHRSST-SSTsubskin-night_SST_HR_NRT-MED-v02.0-fv01.0.nc')

"        \ninput_identifiers = ('20170101_m-OGS--BIOL-MedBFM1-MED-b20190402_re-sv04.10.nc','20180101_m-OGS--BIOL-MedBFM1-MED-b20191101_re-sv04.10.nc',\n                     '20170201_m-OGS--BIOL-MedBFM1-MED-b20190402_re-sv04.10.nc','20180201_m-OGS--BIOL-MedBFM1-MED-b20191101_re-sv04.10.nc',\n                     '20170301_m-OGS--BIOL-MedBFM1-MED-b20190402_re-sv04.10.nc','20180301_m-OGS--BIOL-MedBFM1-MED-b20191101_re-sv04.10.nc')    \n"

**Input references**

This is the Sentinel-1 stack catalogue references

In [9]:
input_references = ( 'https://catalog.terradue.com/sentinel1/search?uid=S1A_IW_GRDH_1SDV_20151226T182813_20151226T182838_009217_00D48F_5D5F', 'https://catalog.terradue.com/sentinel1/search?uid=S1A_IW_GRDH_1SDV_20160424T182813_20160424T182838_010967_010769_AA98', 'https://catalog.terradue.com/sentinel1/search?uid=S1A_IW_GRDH_1SDV_20160518T182817_20160518T182842_011317_011291_936E', 'https://catalog.terradue.com/sentinel1/search?uid=S1A_IW_GRDH_1SDV_20160611T182819_20160611T182844_011667_011DC0_391B', 'https://catalog.terradue.com/sentinel1/search?uid=S1A_IW_GRDH_1SDV_20160705T182820_20160705T182845_012017_0128E1_D4EE', 'https://catalog.terradue.com/sentinel1/search?uid=S1A_IW_GRDH_1SDV_20160729T182822_20160729T182847_012367_013456_E8BF', 'https://catalog.terradue.com/sentinel1/search?uid=S1A_IW_GRDH_1SDV_20160822T182823_20160822T182848_012717_013FFE_90AF', 'https://catalog.terradue.com/sentinel1/search?uid=S1A_IW_GRDH_1SDV_20160915T182824_20160915T182849_013067_014B77_1FCD' ) 

**Data path**

This path defines where the data is staged-in. 

In [10]:
data_path = "/workspace/data/CMEMS/SST_MED_SST_L3S_NRT_OBSERVATIONS_010_012_a/data"

In [11]:
output_folder = "/workspace/Better_3rd_phase/Applications/EXT-03-04-01/ewf-ext-03-04-01/src/main/app-resources/notebook/libexec/Output"

In [12]:
temp_folder = '/workspace/Better_3rd_phase/Applications/EXT-03-04-01/ewf-ext-03-04-01/src/main/app-resources/notebook/libexec/Temp'

In [13]:
cropped_output_folder = '/workspace/Better_3rd_phase/Applications/EXT-03-04-01/ewf-ext-03-04-01/src/main/app-resources/notebook/libexec/Output/Crop'

#### Import Modules

In [14]:
import os
import shutil

import sys
import string
import numpy as np
from osgeo import gdal, ogr, osr
from shapely.wkt import loads
import datetime

import pdb

#### Auxiliary methods

In [15]:
# remove contents of a given folder
# used to clean a temporary folder
def rm_cfolder(folder):
    #folder = '/path/to/folder'
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)
            

def crop_image(input_image, polygon_wkt, output_path, product_type=None):
    
    dataset = None
        
    if input_image.startswith('ftp://') or input_image.startswith('http'):
        try:
            dataset = gdal.Open('/vsigzip//vsicurl/%s' % input_image)
        except Exception as e:
            print(e)
    elif '.nc' in input_image:
        dataset = gdal.Open('NETCDF:' + input_image + ':' + product_type)

    polygon_ogr = ogr.CreateGeometryFromWkt(polygon_wkt)
    envelope = polygon_ogr.GetEnvelope()
    bounds = [envelope[0], envelope[3], envelope[1], envelope[2]]         

    gdal.Translate(output_path, dataset, outputType=gdal.GDT_Int16, projWin=bounds, projWinSRS='EPSG:4326')

    dataset = None


    
def write_output_image(filepath, output_matrix, image_format, data_format, mask=None, output_projection=None, output_geotransform=None, no_data_value=None):
    
    driver = gdal.GetDriverByName(image_format)
    out_rows = np.size(output_matrix, 0)
    out_columns = np.size(output_matrix, 1)
    
    
    if mask is not None and mask is not 0:
        # TODO: check if output folder exists
        output = driver.Create(filepath, out_columns, out_rows, 2, data_format)
        mask_band = output.GetRasterBand(2)
        mask_band.WriteArray(mask)
        if no_data_value is not None:
            output_matrix[mask > 0] = no_data_value
    else:
        output = driver.Create(filepath, out_columns, out_rows, 1, data_format)
    
    if output_projection is not None:
        output.SetProjection(output_projection)
    if output_geotransform is not None:
        output.SetGeoTransform(output_geotransform)
    
    raster_band = output.GetRasterBand(1)
    if no_data_value is not None:
        raster_band.SetNoDataValue(no_data_value)
    raster_band.WriteArray(output_matrix)
    
    if filepath is None:
        print  "filepath"
    if output is None:
        print  "output"
    gdal.Warp(filepath, output, format="GTiff", outputBoundsSRS='EPSG:4326', xRes=output_geotransform[1], yRes=-output_geotransform[5], targetAlignedPixels=True)

    return filepath


    
def calc_max_matrix(mat1, mat2, no_data_value=None):
    
    if no_data_value is not None:
        if not isinstance(mat1, int):
            mat1[(mat1 == no_data_value)] = 0
        if not isinstance(mat2, int):
            mat2[(mat2 == no_data_value)] = 0
    
    return np.where(mat1 > mat2, mat1, mat2)


def matrix_sum(mat1, mat2, no_data_value=None):
    if no_data_value is not None:
        if not isinstance(mat1, int):
            mat1[(mat1 == no_data_value)] = 0
        if not isinstance(mat2, int):
            mat2[(mat2 == no_data_value)] = 0
            
            
    msum = mat1 + mat2
    return msum

def matrix_sum_for_avg(mat1, mat2, mat_n_vals, no_data_value):

    no_data_value_alt = -32768
    
    mat2_0and1s = np.zeros(mat2.shape)
    
    mat2_0and1s[mat2 != no_data_value] = 1
    
    
    mat_n_vals = mat2_0and1s;
    
    
    #msum = mat1
    
    msum = np.copy(mat1)
    
    msum[mat2 != no_data_value] = mat1[mat2 != no_data_value] + mat2[mat2 != no_data_value]
    
    msum = np.where(np.logical_and(mat1 == no_data_value_alt, mat2 != no_data_value), mat2, msum)
    
    msum[np.logical_and(mat1 == no_data_value_alt, mat2 == no_data_value) ] = no_data_value

    
    
    #msum = mat1 + mat2

    return msum, mat_n_vals


def calc_average(matrix_list, n_matrix, no_data_value=None):

    no_data_value_alt = -32768
    
    if not isinstance(matrix_list, list):
        return 0
    
    result = np.copy(matrix_list[0])
    
    result = np.where(result == no_data_value, no_data_value_alt, result)
  
    mat_n_vals = np.zeros(result.shape)
    mat_n_vals[result != no_data_value_alt] = 1
    
    for i in range(1, len(matrix_list)):
     
        result, mat_n_vals_of_sum = matrix_sum_for_avg(result, matrix_list[i], mat_n_vals, no_data_value)
        
        #result = np.copy(result_temp)
        
        mat_n_vals = mat_n_vals + mat_n_vals_of_sum
    

    # to avoid division by 0!!
    mat_n_vals[mat_n_vals == 0] = no_data_value_alt

    result = np.divide(result, mat_n_vals)
    
    # set as no data value pixels that are no data values in all time series
    result[mat_n_vals == no_data_value_alt] = no_data_value
    
    return result

def matrix_sum_for_sigma(mat1, mat2, mat_n_vals, avgmat, no_data_value):

    no_data_value_alt = -32768
    
    mat2_0and1s = np.zeros(mat2.shape)
    
    mat2_0and1s[mat2 != no_data_value] = 1
    
    
    mat_n_vals = mat2_0and1s;
    
    
    #msum = mat1
    
    msum = np.copy(mat1)
    #(matrix_list[0] - avgmat)**2
    msum[mat2 != no_data_value] = mat1[mat2 != no_data_value] + (mat2[mat2 != no_data_value] - avgmat[mat2 != no_data_value])**2
    
    msum = np.where(np.logical_and(mat1 == no_data_value_alt, mat2 != no_data_value), (mat2 - avgmat)**2, msum)
    
    msum[np.logical_and(mat1 == no_data_value_alt, mat2 == no_data_value) ] = no_data_value

    
    
    #msum = mat1 + mat2

    return msum, mat_n_vals


def calc_standarddeviation(matrix_list, n_matrix, avgmat, no_data_value=None):

    no_data_value_alt = -32768
    
    if not isinstance(matrix_list, list):
        return 0
    
    result = (matrix_list[0] - avgmat)**2
    
    
    result = np.where(matrix_list[0] == no_data_value, no_data_value_alt, result)
  
    mat_n_vals = np.zeros(result.shape)
    mat_n_vals[result != no_data_value_alt] = 1
    
    for i in range(1, len(matrix_list)):
     
        result, mat_n_vals_of_sum = matrix_sum_for_sigma(result, matrix_list[i], mat_n_vals, avgmat, no_data_value)
        
        #result = np.copy(result_temp)
        
        mat_n_vals = mat_n_vals + mat_n_vals_of_sum
    

    # to avoid division by 0!!
    mat_n_vals[mat_n_vals == 0] = no_data_value_alt

    result = np.sqrt(np.divide(result, mat_n_vals))
    
    # set as no data value pixels that are no data values in all time series
    result[mat_n_vals == no_data_value_alt] = no_data_value
    
    return result

def get_matrix_list(image_list, product_type):
    projection = None
    geo_transform = None
    no_data = None
    mat_list = []
    for img in image_list:
        dataset = gdal.Open('NETCDF:' + img + ':' + product_type)
        projection = dataset.GetProjection()
        geo_transform = dataset.GetGeoTransform()
        no_data = dataset.GetRasterBand(1).GetNoDataValue()
        product_array = dataset.GetRasterBand(1).ReadAsArray()
        mat_list.append(product_array)
        dataset = None
    return mat_list, projection, geo_transform, no_data

def calculate_anomaly(averagesigma, doy, no_value):
    
    above = doy > averagesigma
    below = doy < averagesigma
    
    above[averagesigma == no_value] = 0
    below[averagesigma == no_value] = 0
    
    return above*1, below*1
    

def write_outputs(product_name, first_date, last_date, averages, standard_deviation, image_format, projection, geo_transform, no_data_value):
    filenames = []
    areaofinterest = area_of_interest['value']
    filenames.append(product_name + '_averages_' + areaofinterest + '_' + first_date + '_' + last_date + '.tif')
    filenames.append(product_name + '_standarddeviation_' + areaofinterest + '_'+ first_date + '_' + last_date + '.tif')

    write_output_image(filenames[0], averages, image_format, gdal.GDT_Int16, None, projection, geo_transform, no_data_value)
    write_output_image(filenames[1], standard_deviation, image_format, gdal.GDT_Int16, None, projection, geo_transform, no_data_value)
    
    return filenames

def isleap(year):
    """Return True for leap years, False for non-leap years."""
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def write_properties_file(output_name, first_date, last_date, region_of_interest):
    
    title = 'Output %s' % output_name
    
    first_date = get_formatted_date(first_date)
    last_date = get_formatted_date(last_date)
    
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (region_of_interest))
        
def get_formatted_date(date_obj):
    date = datetime.datetime.strftime(date_obj, '%Y-%m-%dT00:00:00Z')
    return date


#### Workflow

In [16]:
yoi['value'], start_year['value'], end_year['value']

('2019', '2018', '2019')

In [17]:
from calendar import monthrange

aoi = region_of_interest['value']
yearofinterest = int(yoi['value'])
start_year_int = int(start_year['value'])
end_year_int = int(end_year['value'])
anomaly_year_index = yearofinterest - start_year_int
image_variable_value = image_variable['value']
areaofinterest = area_of_interest['value']

image_variable_value_name = image_variable_value.replace("_", "")
product_path_name = output_folder + '/' + image_variable_value_name

#first_date = os.path.splitext(input_identifiers[0])[0].split('-')[0]
#last_date = os.path.splitext(input_identifiers[-1])[0].split('-')[0]

#input_identifiers.sort()

if input_identifiers[0].find("SST") >=0:     

    doy = 0
    
    product_path_name = output_folder + '/' + image_variable_value_name
    below = None
    above = None
    mask_alltime_novalues = None
    average = None
    projection = None
    geo_transform = None
    no_data = None
    for month in range(1,13):
        getout = False
        for day in range(1, monthrange(yearofinterest, month)[1]+1):
            doy+=1
            file_list = []
            month_and_day = str(month).zfill(2)  + str(day).zfill(2)
            for year in range(start_year_int, end_year_int+1):
                date = str(year) + month_and_day + "000000"
                matching = [s for s in input_identifiers if date in s]
                if len(matching) == 1:
                    file = matching[0]
                    file_list.append(file)
                else:
                    print "file not found on input_identifier => " + date
                    getout = True
                    break
            if getout == True:
                getout = False
                break #continue to try the next day, break next month
            first_year = file_list[0].split('-')[0][:4]
            last_year = file_list[-1].split('-')[0][:4]
            first_date = first_year + str(doy).zfill(3)
            last_date = last_year + str(doy).zfill(3)
            file_list[:] = [data_path + '/' + filename for filename in file_list]
            n_years = end_year_int - start_year_int + 1
            
            if n_years == len(file_list):
                mat_list, projection, geo_transform, no_data = get_matrix_list(file_list, image_variable_value)
                average = calc_average(mat_list, len(file_list), no_data)
                standarddeviations = calc_standarddeviation(mat_list, len(file_list), average, no_data)
                averagesigma = matrix_sum(average, standarddeviations)
                #averagesigma[average == no_data] = no_data
                above_mat, below_mat = calculate_anomaly(averagesigma, mat_list[anomaly_year_index], no_data)
                if (above is None) or (below is None) or (mask_alltime_novalues is None):
                    
                    above = np.zeros(above_mat.shape)
                    below = np.zeros(below_mat.shape)
                    
                    mask_alltime_novalues = (below == 0)
                    
                above += above_mat
                below += below_mat
                
                mask_alltime_novalues = np.logical_and(mask_alltime_novalues, (average == no_data))
                
                for i, file in enumerate(file_list):
                    cropped_file = cropped_output_folder + '/'+ file.split('/')[-1]
                    print cropped_file
                    crop_image(file, aoi, cropped_file, image_variable_value)
                    file_list[i] = cropped_file + '.tif'

                
                files = write_outputs(product_path_name, first_date, last_date, average, standarddeviations, 'GTiff', projection, geo_transform, no_data)
                for output_name in files:
                    firstdate_obj = datetime.datetime.strptime(first_date, "%Y%j").date()
                    lastdate_obj = datetime.datetime.strptime(last_date, "%Y%j").date()
                    write_properties_file(output_name, firstdate_obj, lastdate_obj, region_of_interest)
            else:
                print "error" + str(len(file_list))
    #    if getout == True:
    #        break
else:
    product_path_name = output_folder + '/' + image_variable_value_name
    below = None
    above = None
    mask_alltime_novalues = None
    average = None
    projection = None
    geo_transform = None
    no_data = None
    for month in range(1,13):
        file_list = []
        getout = False
        for year in range(start_year_int, end_year_int+1):
            date = str(year) + str(month).zfill(2) + '01'
            matching = [s for s in input_identifiers if date in s]
            if len(matching) == 1:
                file = matching[0]
                file_list.append(file)
            else:
                print "file not found on input_identifier => " + date
                getout = True
                break
        if getout == True:
            getout = False
            break #continue to try the next day, break next month
        first_year = file_list[0].split('-')[0][:4]
        last_year = file_list[-1].split('-')[0][:4]
        first_date = first_year + str(month).zfill(2)
        last_date = last_year + str(month).zfill(2)
        file_list[:] = [data_path + '/' + filename for filename in file_list]
        n_years = end_year_int - start_year_int + 1
        
        #meter aqui
        
        if n_years == len(file_list):
            mat_list, projection, geo_transform, no_data = get_matrix_list(file_list, image_variable_value)
            
                    
            average = calc_average(mat_list, len(file_list), no_data)
            standarddeviations = calc_standarddeviation(mat_list, len(file_list), average, no_data)
            averagesigma = matrix_sum(average, standarddeviations)
            #averagesigma[average == no_data] = no_data
            above_mat, below_mat = calculate_anomaly(averagesigma, mat_list[anomaly_year_index], no_data)
            if (above is None) or (below is None) or (mask_alltime_novalues is None):
                
                above = np.zeros(above_mat.shape)
                below = np.zeros(below_mat.shape)
                
                mask_alltime_novalues = (below == 0)
                
            above += above_mat
            below += below_mat
            
            mask_alltime_novalues = np.logical_and(mask_alltime_novalues, (average == no_data))
            
            for i, file in enumerate(file_list):
                    cropped_file = cropped_output_folder + '/'+ file.split('/')[-1]
                    print cropped_file
                    crop_image(file, aoi, cropped_file, image_variable_value)
                    file_list[i] = cropped_file + '.tif'
            
            files = write_outputs(product_path_name, first_date, last_date, average, standarddeviations, 'GTiff', projection, geo_transform, no_data)
            for output_name in files:
                firstdate_obj = datetime.datetime.strptime(first_date, "%Y%j").date()
                lastdate_obj = datetime.datetime.strptime(last_date, "%Y%j").date()
                write_properties_file(output_name, first_date, last_date, region_of_interest)
        else:
            print "error" + str(len(file_list))
    #    if getout == True:
    #        break


above[mask_alltime_novalues] = no_data
below[mask_alltime_novalues] = no_data

file = write_output_image(product_path_name + '_above_' + areaofinterest + '_' + first_year + '_' + last_year + '.tif', above, 'GTiff', gdal.GDT_Int16, None, projection, geo_transform, no_data)
firstdate_obj = datetime.datetime.strptime(first_year, "%Y").date()
lastdate_obj = datetime.datetime.strptime(last_year, "%Y").date()
write_properties_file(file, firstdate_obj, lastdate_obj, region_of_interest)
write_output_image(product_path_name + '_below_' + areaofinterest + '_' + first_year + '_' + last_year + '.tif', below, 'GTiff', gdal.GDT_Int16, None, projection, geo_transform, no_data)
write_properties_file(output_name, firstdate_obj, lastdate_obj, region_of_interest)

/workspace/Better_3rd_phase/Applications/EXT-03-04-01/ewf-ext-03-04-01/src/main/app-resources/notebook/libexec/Output/Crop/20180101000000-GOS-L3S_GHRSST-SSTsubskin-night_SST_HR_NRT-MED-v02.0-fv01.0.nc
/workspace/Better_3rd_phase/Applications/EXT-03-04-01/ewf-ext-03-04-01/src/main/app-resources/notebook/libexec/Output/Crop/20190101000000-GOS-L3S_GHRSST-SSTsubskin-night_SST_HR_NRT-MED-v02.0-fv01.0.nc
/workspace/Better_3rd_phase/Applications/EXT-03-04-01/ewf-ext-03-04-01/src/main/app-resources/notebook/libexec/Output/Crop/20180102000000-GOS-L3S_GHRSST-SSTsubskin-night_SST_HR_NRT-MED-v02.0-fv01.0.nc
/workspace/Better_3rd_phase/Applications/EXT-03-04-01/ewf-ext-03-04-01/src/main/app-resources/notebook/libexec/Output/Crop/20190102000000-GOS-L3S_GHRSST-SSTsubskin-night_SST_HR_NRT-MED-v02.0-fv01.0.nc
/workspace/Better_3rd_phase/Applications/EXT-03-04-01/ewf-ext-03-04-01/src/main/app-resources/notebook/libexec/Output/Crop/20180103000000-GOS-L3S_GHRSST-SSTsubskin-night_SST_HR_NRT-MED-v02.0-fv01.